In [5]:

import os
from IPython.display import display, clear_output
from ipywidgets import widgets
import pandas as pd
from yfiles_jupyter_graphs import GraphWidget

def select_input_folder(base_path="./output"):
    folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    if not folders:
        raise ValueError(f"No folders found in {base_path}")
    
    dropdown = widgets.Dropdown(
        options=folders,
        description='Select folder:',
        disabled=False,
    )
    
    output = widgets.Output()
    
    def on_change(change):
        with output:
            clear_output()
            print(f"Selected folder: {change.new}")
            process_selected_folder(change.new)
    
    dropdown.observe(on_change, names='value')
    
    display(dropdown, output)

def process_selected_folder(selected_folder):
    INPUT_DIR = os.path.join("./output", selected_folder, "artifacts")
    LANCEDB_URI = f"{INPUT_DIR}/"

    COMMUNITY_REPORT_TABLE = "create_final_community_reports"
    ENTITY_TABLE = "create_final_nodes"
    ENTITY_EMBEDDING_TABLE = "create_final_entities"
    RELATIONSHIP_TABLE = "create_final_relationships"
    COVARIATE_TABLE = "create_final_covariates"
    TEXT_UNIT_TABLE = "create_final_text_units"
    COMMUNITY_LEVEL = 2

    # read nodes table to get community and degree data
    entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
    entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")
    relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")

    w = create_graph_widget(entity_df, relationship_df)
    display(w)

def create_graph_widget(entity_df, relationship_df):
    w = GraphWidget()
    w.directed = True
    w.nodes = convert_entities_to_dicts(entity_df)
    w.edges = convert_relationships_to_dicts(relationship_df)

    w.node_label_mapping = "title"
    w.node_color_mapping = lambda node: community_to_color(node["properties"]["community"])
    w.edge_color_mapping = lambda edge: community_to_color(edge_to_source_community(edge, w.nodes))
    w.node_scale_factor_mapping = lambda node: 0.5 + node["properties"]["size"] * 1.5 / 20
    w.edge_thickness_factor_mapping = "weight"

    w.radial_layout()
    return w

def convert_entities_to_dicts(df):
    nodes_dict = {}
    for _, row in df.iterrows():
        node_id = row["title"]
        if node_id not in nodes_dict:
            nodes_dict[node_id] = {
                "id": node_id,
                "properties": row.to_dict(),
            }
    return list(nodes_dict.values())

def convert_relationships_to_dicts(df):
    relationships = []
    for _, row in df.iterrows():
        relationships.append({
            "start": row["source"],
            "end": row["target"],
            "properties": row.to_dict(),
        })
    return relationships

def community_to_color(community):
    colors = [
        "crimson", "darkorange", "indigo", "cornflowerblue",
        "cyan", "teal", "green",
    ]
    return colors[int(community) % len(colors)] if community is not None else "lightgray"

def edge_to_source_community(edge, nodes):
    source_node = next(
        (entry for entry in nodes if entry["properties"]["title"] == edge["start"]),
        None,
    )
    source_node_community = source_node["properties"]["community"] if source_node else None
    return source_node_community if source_node_community is not None else None

# Kullanım
select_input_folder()


Dropdown(description='Select folder:', options=('comert_dinleme', '123'), value='comert_dinleme')

Output()